In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
import math
try: from sklearn.model_selection import train_test_split
except: from sklearn.cross_validation import train_test_split

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from sklearn import datasets
print(sklearn.__version__)
print(tf.__version__)
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import matplotlib
import math
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
font = fm.FontProperties(family = 'Gill Sans', fname = '/Library/Fonts/GillSans.ttc')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
def hide_spines(intx=False,inty=False,logscale=0):
    """Hides the top and rightmost axis spines from view for all active
    figures and their respective axes."""

    # Retrieve a list of all current figures.
    figures = [x for x in matplotlib._pylab_helpers.Gcf.get_all_fig_managers()]
    if (plt.gca().get_legend()):
        plt.setp(plt.gca().get_legend().get_texts(), fontproperties=font) 
    for figure in figures:
        # Get all Axis instances related to the figure.
        for ax in figure.canvas.figure.get_axes():
            # Disable spines.
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')
            # Disable ticks.
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
            #ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
            for label in ax.get_xticklabels() :
                label.set_fontproperties(font)
            for label in ax.get_yminorticklabels() :
                label.set_fontproperties(font)
            for label in ax.get_ymajorticklabels() :
                label.set_fontproperties(font)
            #ax.set_xticklabels(ax.get_xticks(), fontproperties = font)
            ax.set_xlabel(ax.get_xlabel(), fontproperties = font)
            ax.set_ylabel(ax.get_ylabel(), fontproperties = font)
            ax.set_title(ax.get_title(), fontproperties = font)
            if (inty):
                ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
            if (intx):
                ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
            if (logscale):
                ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % (math.log(v,10)) )))
def show(nm,a=0,b=0,logscale=0, facecolor=0):
    hide_spines(a,b,logscale)
    #ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
    #plt.yticks([1,1e-2,1e-4,1e-6,1e-8,1e-10,1e-12], labels)
    if (type(facecolor) == str):
        plt.savefig(nm, facecolor=facecolor)
    else:
        plt.savefig(nm);
    plt.show()
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18

fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(test_labels.shape)
x_train = train_images #/255
y_train = train_labels
x_valid = test_images #/255
y_valid = test_labels
w, h = 28, 28
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_train = tf.keras.utils.to_categorical(y_train, 10)
print(x_train.shape)
print(y_valid.shape,y_train.shape)


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64).shuffle(120000)
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
#train_dataset = train_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()


valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(5000) #.shuffle(10000)
valid_dataset = valid_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
#valid_dataset = valid_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
valid_dataset = valid_dataset.repeat()

0.20.3
2.0.0-beta1
(10000,)
(60000, 28, 28, 1)
(10000, 10) (60000, 10)


In [10]:
def trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3):
    densev1_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=(1,1), padding='same')
    ldt = tf.keras.layers.Lambda(lambda x: x*dt)
    diag_lam = tf.keras.layers.Lambda(lambda x: x/(1+dt/epsilon))
    off_lam = tf.keras.layers.Lambda(lambda x: dt*x/epsilon/(1+dt/epsilon))
    tfact = tf.keras.layers.Activation(activation="relu")
    tfa = tf.keras.layers.Add()
    tfm = tf.keras.layers.Subtract()
    for step in range(steps):
        v1 = tfa([v1, ldt(densev1_1(v2))])
        tp = tf.keras.layers.Permute(dims=(2,1,3))(v1)
        v2 = tfm([v2, ldt(densev1_1(tp))]) 
        v2 = diag_lam(v2)
        v_o = tfact(v1)
        v_o =  off_lam(v_o)
        v2 = tfa([v2, v_o]) 
    return v1,v2

def res_block(dt, steps, v1, filters, kernel_size=3):
    densev1_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=(1,1), 
                                       padding='same', activation="relu")
    ldt = tf.keras.layers.Lambda(lambda x: x*dt)
    diag_lam = tf.keras.layers.Lambda(lambda x: x/(1+dt/epsilon))
    tfa = tf.keras.layers.Add()
    for step in range(steps):
        v1 = tfa([v1, ldt(densev1_1(v1))])
    return v1

def baseline_model(x):
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same', activation="relu")(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
    x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(x)
    x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same', activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
    x = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(x))
    outputs = tf.keras.layers.Dense(10,activation="softmax")(x)
    return outputs

In [3]:
#create network
inputs = keras.Input(shape=(28,28,1,), name='MNIST')

output_base = baseline_model(inputs)

baseline = keras.Model(inputs, output_base)
baseline.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
baseline.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MNIST (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        128   

In [4]:
import datetime
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/base_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/baseline_fashion.hdf5', verbose = 1, save_best_only=True)]

history = baseline.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1874/1875 [============================>.] - ETA: 0s - loss: 0.3706 - accuracy: 0.8686
Epoch 00001: val_loss improved from inf to 0.31164, saving model to ./log/baseline_fashion.hdf5
1875/1875 [==============================] - 178s 95ms/step - loss: 0.3706 - accuracy: 0.8686 - val_loss: 0.3116 - val_accuracy: 0.8923
Epoch 2/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2621 - accuracy: 0.9062
Epoch 00002: val_loss did not improve from 0.31164
1875/1875 [==============================] - 173s 92ms/step - loss: 0.2621 - accuracy: 0.9062 - val_loss: 0.3331 - val_accuracy: 0.8879
Epoch 3/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2283 - accuracy: 0.9183
Epoch 00003: val_loss improved from 0.31164 to 0.30431, saving model to ./log/baseline_fashion.hdf5
1875/1875 [==============================] - 187s 100ms/step - loss: 0.2283 - accuracy: 0.9183 -

In [14]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)
v2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.01
steps = 5
dt = 1.0/steps
filters = 64
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)
v2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v2)
v2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v2)
v2 = tf.keras.layers.BatchNormalization()(v2)

#second trans block
filters = 32
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model = keras.Model(inputs=inputs, outputs=outputs)

In [15]:
model.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model.summary()
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/steps5_fashion.hdf5', verbose = 1, save_best_only=True)]
history = model.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_18[0][0]                  
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_20[0][0]                  
                                                                 permute_10[0][0]           

Epoch 1/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.3541 - accuracy: 0.8759
Epoch 00001: val_loss improved from inf to 0.34847, saving model to ./log/steps5_fashion.hdf5
1875/1875 [==============================] - 1475s 787ms/step - loss: 0.3541 - accuracy: 0.8759 - val_loss: 0.3485 - val_accuracy: 0.8804
Epoch 2/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2348 - accuracy: 0.9159
Epoch 00002: val_loss improved from 0.34847 to 0.27232, saving model to ./log/steps5_fashion.hdf5
1875/1875 [==============================] - 1445s 771ms/step - loss: 0.2347 - accuracy: 0.9160 - val_loss: 0.2723 - val_accuracy: 0.9046
Epoch 3/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.1973 - accuracy: 0.9296
Epoch 00003: val_loss improved from 0.27232 to 0.26260, saving model to ./log/steps5_fashion.hdf5
1875/1875 [==============================] - 1465s 781ms/step - loss: 0.1973 - accuracy: 0.9296 - val_loss: 0.2626 - val_accuracy: 0.9092
Epoch

In [12]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.01
steps = 5
dt = 1.0/steps
filters = 64
v1 = res_block(dt, steps, v1, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)

#second trans block
filters = 32
v1 = res_block(dt, steps, v1, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model_res = keras.Model(inputs=inputs, outputs=outputs)

In [16]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)
v2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.1
steps = 5
dt = 1.0/steps
filters = 64
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)
v2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v2)
v2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v2)
v2 = tf.keras.layers.BatchNormalization()(v2)

#second trans block
filters = 32
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model01 = keras.Model(inputs=inputs, outputs=outputs)

In [17]:
model01.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model01.summary()
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/steps5e01_fashion.hdf5', verbose = 1, save_best_only=True)]
history = model01.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_25[0][0]                  
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_27[0][0]                  
                                                                 permute_20[0][0]           

Epoch 1/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.3464 - accuracy: 0.8786
Epoch 00001: val_loss improved from inf to 0.30372, saving model to ./log/steps5e01_fashion.hdf5
1875/1875 [==============================] - 1430s 763ms/step - loss: 0.3463 - accuracy: 0.8786 - val_loss: 0.3037 - val_accuracy: 0.8956
Epoch 2/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2347 - accuracy: 0.9167
Epoch 00002: val_loss improved from 0.30372 to 0.28047, saving model to ./log/steps5e01_fashion.hdf5
1875/1875 [==============================] - 1444s 770ms/step - loss: 0.2347 - accuracy: 0.9168 - val_loss: 0.2805 - val_accuracy: 0.9015
Epoch 3/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.1999 - accuracy: 0.9286
Epoch 00003: val_loss improved from 0.28047 to 0.26473, saving model to ./log/steps5e01_fashion.hdf5
1875/1875 [==============================] - 1424s 760ms/step - loss: 0.1999 - accuracy: 0.9286 - val_loss: 0.2647 - val_accuracy: 0.9

In [13]:
import datetime
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/res_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/resnet_fashion.hdf5', verbose = 1, save_best_only=True)]

model_res.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model_res.summary()
history = model_res.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_13[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_14[0][0]                  
                                                                 add_2[0][0]                

Epoch 5/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.1688 - accuracy: 0.9391
Epoch 00005: val_loss improved from 0.25714 to 0.24830, saving model to ./log/resnet_fashion.hdf5
1875/1875 [==============================] - 696s 371ms/step - loss: 0.1688 - accuracy: 0.9391 - val_loss: 0.2483 - val_accuracy: 0.9160

history dict: {'loss': [0.38524749684232024, 0.2585924125595561, 0.21855817410375292, 0.19107857787049312, 0.1687498599591018], 'accuracy': [0.86518645, 0.9072672, 0.9215915, 0.9314801, 0.93910086], 'val_loss': [0.36304710805416107, 0.2771250158548355, 0.25714211165905, 0.26253098994493484, 0.24829839169979095], 'val_accuracy': [0.8786, 0.9011, 0.9133, 0.9125, 0.916]}


In [18]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)
v2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.001
steps = 5
dt = 1.0/steps
filters = 64
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)
v2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v2)
v2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v2)
v2 = tf.keras.layers.BatchNormalization()(v2)

#second trans block
filters = 32
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model0p001 = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
import datetime
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/steps5e0p001_fashion_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/steps5e0p001_fashion.hdf5', verbose = 1, save_best_only=True)]

model0p001.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model0p001.summary()
history = model0p001.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_32[0][0]                  
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_34[0][0]                  
                                                                 permute_30[0][0]           

Epoch 1/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.3544 - accuracy: 0.8743
Epoch 00001: val_loss improved from inf to 0.28328, saving model to ./log/steps5e0p001_fashion.hdf5
1875/1875 [==============================] - 1481s 790ms/step - loss: 0.3543 - accuracy: 0.8743 - val_loss: 0.2833 - val_accuracy: 0.8975
Epoch 2/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2294 - accuracy: 0.9182
Epoch 00002: val_loss improved from 0.28328 to 0.26979, saving model to ./log/steps5e0p001_fashion.hdf5
1875/1875 [==============================] - 1491s 795ms/step - loss: 0.2293 - accuracy: 0.9182 - val_loss: 0.2698 - val_accuracy: 0.9051
Epoch 3/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.1937 - accuracy: 0.9306
Epoch 00003: val_loss improved from 0.26979 to 0.25092, saving model to ./log/steps5e0p001_fashion.hdf5
1875/1875 [==============================] - 1460s 779ms/step - loss: 0.1937 - accuracy: 0.9306 - val_loss: 0.2509 - val_accu

In [20]:
# 10 steps res
init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
steps = 10
dt = 1.0/steps
filters = 64
v1 = res_block(dt, steps, v1, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)

#second trans block
filters = 32
v1 = res_block(dt, steps, v1, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model_res_10 = keras.Model(inputs=inputs, outputs=outputs)

In [21]:
import datetime
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/res_10_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/resnet_fashion_10.hdf5', verbose = 1, save_best_only=True)]

model_res_10.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model_res_10.summary()
history = model_res_10.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_39[0][0]                  
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_40[0][0]                  
                                                                 add_10[0][0]               

Epoch 1/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.3804 - accuracy: 0.8662
Epoch 00001: val_loss improved from inf to 0.36960, saving model to ./log/resnet_fashion_10.hdf5
1875/1875 [==============================] - 1544s 823ms/step - loss: 0.3804 - accuracy: 0.8662 - val_loss: 0.3696 - val_accuracy: 0.8808
Epoch 2/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2530 - accuracy: 0.9106
Epoch 00002: val_loss improved from 0.36960 to 0.29176, saving model to ./log/resnet_fashion_10.hdf5
1875/1875 [==============================] - 1423s 759ms/step - loss: 0.2530 - accuracy: 0.9106 - val_loss: 0.2918 - val_accuracy: 0.8943
Epoch 3/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.2145 - accuracy: 0.9227
Epoch 00003: val_loss improved from 0.29176 to 0.26512, saving model to ./log/resnet_fashion_10.hdf5
1875/1875 [==============================] - 1422s 759ms/step - loss: 0.2145 - accuracy: 0.9227 - val_loss: 0.2651 - val_accuracy: 0.9

In [22]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)
v2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.01
steps = 10
dt = 1.0/steps
filters = 64
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)
v2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v2)
v2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v2)
v2 = tf.keras.layers.BatchNormalization()(v2)

#second trans block
filters = 32
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model_10 = keras.Model(inputs=inputs, outputs=outputs)

In [23]:
model_10.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
model_10.summary()
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/steps10_e0p01_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/steps10_e0p01_fashion.hdf5', verbose = 1, save_best_only=True)]
history = model_10.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MNIST (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 28, 28, 2)    20          MNIST[0][0]                      
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 28, 28, 64)   1216        conv2d_44[0][0]                  
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 28, 28, 64)   36928       conv2d_46[0][0]                  
                                                                 permute_40[0][0]           

Epoch 1/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.3467 - accuracy: 0.8790
Epoch 00001: val_loss improved from inf to 0.31274, saving model to ./log/steps10_e0p01_fashion.hdf5
1875/1875 [==============================] - 2794s 1s/step - loss: 0.3466 - accuracy: 0.8790 - val_loss: 0.3127 - val_accuracy: 0.8913
Epoch 2/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.2256 - accuracy: 0.9189
Epoch 00002: val_loss improved from 0.31274 to 0.25878, saving model to ./log/steps10_e0p01_fashion.hdf5
1875/1875 [==============================] - 2772s 1s/step - loss: 0.2256 - accuracy: 0.9189 - val_loss: 0.2588 - val_accuracy: 0.9091
Epoch 3/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.1900 - accuracy: 0.9318
Epoch 00003: val_loss improved from 0.25878 to 0.24022, saving model to ./log/steps10_e0p01_fashion.hdf5
1875/1875 [==============================] - 2765s 1s/step - loss: 0.1900 - accuracy: 0.9318 - val_loss: 0.2402 - val_accuracy: 

In [24]:

init_split = tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding='same')(inputs)
v1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)
v2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same')(init_split)

#first trans block
epsilon = 0.001
steps = 10
dt = 1.0/steps
filters = 64
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)

#pool, conv to 32 filters and normalize
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v1)
v1 = tf.keras.layers.BatchNormalization()(v1)
v2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v2)
v2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same')(v2)
v2 = tf.keras.layers.BatchNormalization()(v2)

#second trans block
filters = 32
v1, v2 = trans_block(dt, epsilon, steps, v1, v2, filters, kernel_size=3)


#pool and flatten
v1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(v1)
v1 = tf.keras.layers.Flatten()(tf.keras.layers.Dropout(0.01)(v1))
dense_final = tf.keras.layers.Dense(10,activation="softmax")
outputs = dense_final(v1)


model_10_0p0001 = keras.Model(inputs=inputs, outputs=outputs)

In [25]:
model_10_0p0001.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=['accuracy'])
#model_10_0p0001.summary()
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/steps10_e0p001_{}'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
keras.callbacks.ModelCheckpoint(filepath='./log/steps10_e0p001_fashion.hdf5', verbose = 1, save_best_only=True)]
history = model_10_0p0001.fit(train_dataset,
                    epochs=5,steps_per_epoch=120000//64,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(valid_dataset), validation_steps = 4,
                   callbacks=callbacks)

print('\nhistory dict:', history.history)

Epoch 1/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.3436 - accuracy: 0.8791
Epoch 00001: val_loss improved from inf to 0.29981, saving model to ./log/steps10_e0p001_fashion.hdf5
1875/1875 [==============================] - 2735s 1s/step - loss: 0.3436 - accuracy: 0.8791 - val_loss: 0.2998 - val_accuracy: 0.8959
Epoch 2/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.2273 - accuracy: 0.9184
Epoch 00002: val_loss improved from 0.29981 to 0.27073, saving model to ./log/steps10_e0p001_fashion.hdf5
1875/1875 [==============================] - 2687s 1s/step - loss: 0.2273 - accuracy: 0.9185 - val_loss: 0.2707 - val_accuracy: 0.9070
Epoch 3/5
1874/1875 [============================>.] - ETA: 1s - loss: 0.1923 - accuracy: 0.9316
Epoch 00003: val_loss improved from 0.27073 to 0.24420, saving model to ./log/steps10_e0p001_fashion.hdf5
1875/1875 [==============================] - 2698s 1s/step - loss: 0.1923 - accuracy: 0.9315 - val_loss: 0.2442 - val_accurac

In [26]:
120000//64

1875